# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [9]:
# desde la consola:
# pip install tweepy,
# pip install python-dotenv
# pip install pandas
# pip install matplotlib
# pip install seaborn

# librerias
import os
from dotenv import load_dotenv
import tweepy
import requests
import pandas
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# cargo .env
load_dotenv()

# cargo datos de acceso desde archivo .env
consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')
access_token = os.environ.get('ACCESS_TOKEN')
access_token_secret = os.environ.get('ACCESS_TOKEN_SECRET')
bearer_token = os.environ.get('BEARER_TOKEN')

In [ ]:
# función para conectarse a Tweeter
client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

In [ ]:
# defino query
query = '#100daysofcode (pandas OR python) -is:retweet'

# traigo 100 tweets más recientes
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id','created_at','lang'],
                                     max_results=100)

In [ ]:
# Save data as dictionary
tweets_dict = tweets.json()

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data)
df.head()

# save df
df.to_csv("coding-tweets.csv")

In [ ]:
# funcion para saber si un texto está dentro de un tweet
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)

    if match:
        return True
    return False

In [ ]:
### busco cuantas veces figura las palabras "pandas" y "python" en los tweets

# inicializo lista
[pandas, python] = [0, 0]

# itero filas del df
for index, row in df.iterrows():
    pandas += word_in_text('pandas', row['text'])
    python += word_in_text('python', row['text'])

In [ ]:
# grafico resultados

# Set seaborn style
sns.set(color_codes=True)

# Create a list of labels:cd
cd = ['pandas', 'python']

# Plot the bar chart
ax = sns.barplot(cd, [pandas, python])
ax.set(ylabel="count")
plt.show()